In [8]:
from gurobipy import Model, GRB, quicksum

P = {
    1: {1: 2, 2: 2, 3: 1, 4: 0, 5: 3, 6: 3, 7: 4, 8: 3, 9: 5, 10: 5},
    2: {1: 5, 2: 5, 3: 1, 4: 2, 5: 1, 6: 5, 7: 4, 8: 3, 9: 2, 10: 2},
    3: {1: 1, 2: 1, 3: 5, 4: 1, 5: 1, 6: 1, 7: 0, 8: 1, 9: 1, 10: 2},
    4: {1: 4, 2: 3, 3: 2, 4: 0, 5: 1, 6: 1, 7: 4, 8: 4, 9: 4, 10: 4},
    5: {1: 2, 2: 2, 3: 2, 4: 1, 5: 1, 6: 2, 7: 3, 8: 3, 9: 4, 10: 5},
    6: {1: 2, 2: 2, 3: 3, 4: 4, 5: 1, 6: 0, 7: 1, 8: 1, 9: 2, 10: 3},
    7: {1: 2, 2: 1, 3: 1, 4: 1, 5: 5, 6: 4, 7: 4, 8: 5, 9: 4, 10: 4}
}

Q = {
    1: {1: 3, 2: 2, 3: 4, 4: 1, 5: 5, 6: 3, 7: 2, 8: 5, 9: 3, 10: 1},
    2: {1: 5, 2: 4, 3: 5, 4: 3, 5: 1, 6: 2, 7: 1, 8: 1, 9: 2, 10: 4},
    3: {1: 3, 2: 5, 3: 1, 4: 1, 5: 2, 6: 2, 7: 1, 8: 1, 9: 1, 10: 1},
    4: {1: 2, 2: 0, 3: 0, 4: 1, 5: 2, 6: 1, 7: 1, 8: 0, 9: 0, 10: 0},
    5: {1: 1, 2: 2, 3: 3, 4: 2, 5: 1, 6: 1, 7: 2, 8: 1, 9: 3, 10: 2},
    6: {1: 1, 2: 2, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: 0, 9: 0, 10: 0},
    7: {1: 1, 2: 2, 3: 2, 4: 2, 5: 1, 6: 1, 7: 1, 8: 5, 9: 1, 10: 1}
}

# Now, integrate these into the complete Gurobi model code.


from gurobipy import Model, GRB, quicksum

# Create a new model
m = Model("course_scheduling")

# Indices and sets
I = range(1, 8)    # Set of instructors (1 to 7)
J = range(1, 11)   # Set of courses (1 to 10)
T = range(1, 11)   # Set of time slots (1 to 10)
K = 3              # Number of classrooms

# Decision variables
x = m.addVars(I, J, vtype=GRB.BINARY, name="x")
y = m.addVars(I, T, vtype=GRB.BINARY, name="y")

# Objective function
m.setObjective(0.6 * quicksum(P[i][j] * x[i, j] for i in I for j in J) +
                0.4 * quicksum(Q[i][t] * y[i, t] for i in I for t in T),
                sense=GRB.MAXIMIZE)

# Constraints
# Each instructor is assigned to exactly one course
#for i in I:
#    m.addConstr(quicksum(x[i, j] for j in J) == 1, f"OneCourse_{i}")
    
for j in J:
    m.addConstr(quicksum(x[i, j] for i in I) == 1, f"UniqueInstructor_{j}")
    
# Each instructor can teach a maximum of two courses
for i in I:
    m.addConstr(quicksum(x[i, j] for j in J) <= 2, f"MaxTwoCourses_{i}")
    
# Instructor-course consistency
for i in I:
    for j in J:
        m.addConstr(quicksum(x[i, j] for j in J) == quicksum(y[i, t] for t in T), f"Consistency_{i}_{j}")

# Room capacity constraint
for t in T:
    m.addConstr(quicksum(y[i, t] for i in I) <= K, f"RoomCap_{t}")

# Solve model
m.optimize()

# Print solution
if m.status == GRB.OPTIMAL:
    print("Optimal solution found:\n")
    for i in I:
        for j in J:
            if x[i, j].X > 0.5:
                print(f"Instructor {i} teaches Course {j}")
    for i in I:
        for t in T:
            if y[i, t].X > 0.5:
                print(f"Instructor {i} teaches in Time Slot {t}")
else:
    print("No optimal solution found.")


Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (win64 - Windows 10.0 (19045.2))

CPU model: Intel(R) Core(TM) i5-7300HQ CPU @ 2.50GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 4 logical processors, using up to 4 threads

Optimize a model with 97 rows, 140 columns and 1610 nonzeros
Model fingerprint: 0x4eac732e
Variable types: 0 continuous, 140 integer (140 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [4e-01, 3e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 3e+00]
Presolve removed 63 rows and 0 columns
Presolve time: 0.04s
Presolved: 34 rows, 140 columns, 350 nonzeros
Variable types: 0 continuous, 140 integer (140 binary)
Found heuristic solution: objective 36.0000000

Root relaxation: objective 4.380000e+01, 60 iterations, 0.04 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

